# Santos Dumont (SD) - Numba CPU MPI B710

In [2]:
%%writefile numbampib710.py
import numpy as np
from time import time
from numba import njit, set_num_threads, get_num_threads
from mpi4py import MPI   

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', parallel=True, fastmath=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1] = (aold[1:-1,1:-1]/2.0
        +(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2])/8.0)

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations
# initialize three heat sources
nsources     = 3    # sources of energy
sources      = np.zeros((nsources, 2), np.int16)
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# main routine
comm    = MPI.COMM_WORLD
mpisize = comm.size
mpirank = comm.rank
if not mpirank : t0 = -time()

# determine my coordinates (x,y)
pdims = MPI.Compute_dims(mpisize, 2)
px    = pdims[0]
py    = pdims[1]
rx    = mpirank % px
ry    = mpirank // px

# determine my four neighbors
north = (ry - 1) * px + rx
if (ry - 1) < 0 :
    north = MPI.PROC_NULL
south = (ry + 1) * px + rx
if (ry + 1) >= py :
    south = MPI.PROC_NULL
west = ry * px + rx - 1
if (rx - 1) < 0 :
    west = MPI.PROC_NULL
east = ry * px + rx + 1
if (rx + 1) >= px :
    east = MPI.PROC_NULL

# decompose the domain
bx   = n // px          # block size in x
by   = n // py          # block size in y
offx = rx * bx + 1      # offset in x
offy = ry * by + 1      # offset in y

# sources in my area, local to my rank
locnsources = 0
locsources  = np.empty((nsources, 2), np.int16)

# determine which sources are in my patch
for i in range(nsources) :
    locx = sources[i, 0] - offx
    locy = sources[i, 1] - offy
    if(locx >= 0 and locx <= bx and locy >= 0 and locy <= by) :
        locsources[locnsources, 0] = locx
        locsources[locnsources, 1] = locy
        locnsources += 1

# working arrays with 1-wide halo zones
anew = np.zeros((bx+2, by+2), np.float64)
aold = np.zeros((bx+2, by+2), np.float64)

# system total heat
rheat = np.zeros(1, np.float64)
bheat = np.zeros(1, np.float64)

for _ in range(0, niters, 2) :
    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(aold[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(aold[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(aold[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(aold[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        aold[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        aold[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        aold[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        aold[1:bx+1, 0] = r4.wait()

    # update grid
    kernel(anew, aold)

    # refresh heat sources
    anew[locsources[:locnsources, 0], locsources[:locnsources, 1]] += energy

    # exchange data with neighbors
    if north != MPI.PROC_NULL :
        r1=comm.irecv(source=north, tag=1)
        s1=comm.isend(anew[1, 1:bx+1], dest=north, tag=1)
    if south != MPI.PROC_NULL :
        r2=comm.irecv(source=south, tag=1)
        s2=comm.isend(anew[bx, 1:bx+1], dest=south, tag=1)
    if east != MPI.PROC_NULL :
        r3 = comm.irecv(source=east, tag=1)
        s3 = comm.isend(anew[1:bx+1, bx], dest=east, tag=1)
    if west != MPI.PROC_NULL :
        r4 = comm.irecv(source=west, tag=1)
        s4 = comm.isend(anew[1:bx+1, 1], dest=west, tag=1)
    # wait
    if north != MPI.PROC_NULL :
        s1.wait()
        anew[0, 1:bx+1] = r1.wait()
    if south != MPI.PROC_NULL :
        s2.wait()
        anew[bx+1, 1:bx+1] = r2.wait()
    if east != MPI.PROC_NULL :
        s3.wait()
        anew[1:bx+1, bx+1] = r3.wait()
    if west != MPI.PROC_NULL :
        s4.wait
        anew[1:bx+1, 0] = r4.wait()

    # update grid
    kernel(aold, anew)

    # refresh heat sources
    aold[locsources[:locnsources, 0], locsources[:locnsources, 1]] += energy 

# get final heat in the system
bheat[0] = np.sum(aold[1:-1, 1:-1])
comm.Reduce(bheat, rheat)

if not mpirank :
    t0 += time()
    print(f"Heat: {rheat[0]:.4f}", end=" | ")
    print(f"Time: {t0:.4f}", end=" | ")
    print(f"MPISize: {mpisize}")

Writing numbampib710.py


### Testa a execução

In [15]:
%%bash
module unload anaconda3
module unload openmpi/gnu/4.1.4

In [9]:
%%bash
# module load openmpi/gnu/4.1.4
# module load anaconda3
module load oneapi

 
:: initializing oneAPI environment ...
   bash: BASH_VERSION = 4.2.46(2)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: clck -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: intelpython -- latest
:: ipp -- latest
:: ippcp -- latest
:: ipp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vpl -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 


ATENTION: It is necessery to run the command 'source /scratch/app/oneapi/2022.2/setvars.sh' to configure the environment! This module does not perform automatic environment configuration


In [16]:
! source /scratch/app/oneapi/2022.2/setvars.sh

 
:: initializing oneAPI environment ...
   bash: BASH_VERSION = 4.2.46(2)-release
   args: Using "$@" for setvars.sh arguments: 
:: advisor -- latest
:: ccl -- latest
:: clck -- latest
:: compiler -- latest
:: dal -- latest
:: debugger -- latest
:: dev-utilities -- latest
:: dnnl -- latest
:: dpcpp-ct -- latest
:: dpl -- latest
:: inspector -- latest
:: intelpython -- latest
:: ipp -- latest
:: ippcp -- latest
:: ipp -- latest
:: itac -- latest
:: mkl -- latest
:: mpi -- latest
:: tbb -- latest
:: vpl -- latest
:: vtune -- latest
:: oneAPI environment initialized ::
 


In [1]:
! conda env list

# conda environments:
#
                      *  /scratch/ampemi/eduardo.miranda2/env
base                     /scratch/app/anaconda3/2020.11



In [13]:
! conda activate base


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [2]:
! mpiexec -n 1 python numbampib710.py

Traceback (most recent call last):
  File "/prj/ampemi/eduardo.miranda2/tama/numbampib710.py", line 1, in <module>
    import numpy as np
ModuleNotFoundError: No module named 'numpy'


Testa com 16 processos:

In [3]:
%%bash
module load intel_psxe/2020
source /opt/intel/parallel_studio_xe_2020/intelpython3/etc/profile.d/conda.sh
unset I_MPI_PMI_LIBRARY
mpiexec -n 16 python numbampib710.py

Heat: 750.0000 | Time: 1.1839 | MPISize: 16


### Copia arquivo com código python para /scratch

In [5]:
! cp  numbampib710.py  /scratch${PWD#/prj}

### Arquivo de lote do Slurm

In [6]:
%%writefile numbampi.srm
#!/bin/bash
#SBATCH --ntasks=96            #Total de tarefas
#SBATCH --job-name numbampi    #Nome do job, 8 caracteres
#SBATCH --partition cpu_dev    #Fila (partition) a ser utilizada
#SBATCH --time=00:01:00        #Tempo max. de execução
#SBATCH --exclusive            #Utilização exclusiva dos nós

echo '- Job ID:' $SLURM_JOB_ID
echo '- Tarefas por no:' $SLURM_NTASKS_PER_NODE
echo '- Qtd. de nos:' $SLURM_JOB_NUM_NODES
echo '- Tot. de tarefas:' $SLURM_NTASKS
echo '- Nos alocados:' $SLURM_JOB_NODELIST
nodeset -e $SLURM_JOB_NODELIST

#Modulos
module load intel_psxe/2020
source /opt/intel/parallel_studio_xe_2020/intelpython3/etc/profile.d/conda.sh

#Entra no diretório de trabalho
cd /scratch${PWD#/prj}

#Executavel
EXEC='python numbampib710.py'

#Dispara a execucao
srun --mpi=pmi2  -n $SLURM_NTASKS  $EXEC

Writing numbampi.srm


## Envia para a fila de execução dev

In [7]:
%%bash
sbatch numbampi.srm
squeue --user $(whoami) -h -r | wc -l
squeue --partition=cpu_dev -h -r | wc -l
squeue --start --name=numbampi --format "%S %.8i %.9P %.5j %.2t %.5M %.5D %.4C"

Submitted batch job 2542360
1
2
START_TIME    JOBID PARTITION  NAME ST  TIME NODES CPUS
N/A  2542360   cpu_dev numba PD  0:00     4   96


Verifica se já executou:

In [8]:
! squeue --start --name=numbampi --format "%S %.8i %.9P %.5j %.2t %.5M %.5D %.4C"

START_TIME    JOBID PARTITION  NAME ST  TIME NODES CPUS


Mostra o arquivo contendo a saída:

In [9]:
! cat /scratch${PWD#/prj}/slurm-2542360.out

- Job ID: 2542360
- Tarefas por no:
- Qtd. de nos: 4
- Tot. de tarefas: 96
- Nos alocados: sdumont[1245-1248]
sdumont1245 sdumont1246 sdumont1247 sdumont1248
Heat: 602.6262 | Time: 3.2165 | MPISize: 96
